In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
#import matplotlib.pyplot as plt
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 5.1 MB/s 


In [6]:
from __future__ import print_function
from __future__ import division
import gc
import os
import sys
import numpy as np
import tensorflow.compat.v1 as tf
from PIL import Image
import argparse
from glob import glob
import time
import natsort
import scipy.io as scio
import tf_slim as slim
import matplotlib.pyplot as plt
#!pip install tf_slim

def cal_psnr(im1, im2): 
    # assert pixel value range is 0-255 and type is uint8
    snr = -20 * np.log10(np.linalg.norm(im1.flatten()-im2.flatten())/np.linalg.norm(im2.flatten()))
    mse = (((im1.astype(np.float) - im2.astype(np.float)) ** 2)/((im2.astype(np.float))**2)).mean()
    psnr = -20 * np.log10(mse)
    return snr


def tf_psnr(im1, im2):
    # assert pixel value range is 0-1
    mse = tf.losses.mean_squared_error(labels=im2 * 255.0, predictions=im1 * 255.0)
    return 10.0 * (tf.log(255.0 ** 2 / mse) / tf.log(10.0))

def upsample_and_sum(x1, x2, output_channels, in_channels, scope=None):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02), name=scope)#0.02
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])
    deconv_output = deconv + x2
    deconv_output.set_shape([None, None, None, output_channels])
    return deconv_output


def FCN(input):
    with tf.variable_scope('fcn'):
        conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv1')
        conv2 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv2')
        conv3 = slim.conv2d(conv2, 32, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3')
        conv4 = slim.conv2d(conv3, 32, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv4')
        conv5 = slim.conv2d(conv4, 1, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv5')
        return conv5

def UNet(input):
    with tf.variable_scope('unet'):
        conv1 = slim.conv2d(input, 64, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv1_1')
        conv1 = slim.conv2d(conv1, 64, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv1_2')
        pool1 = slim.avg_pool2d(conv1, [2, 2], padding='SAME')
        conv2 = slim.conv2d(pool1, 128, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv2_1')
        conv2 = slim.conv2d(conv2, 128, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv2_2')
        conv2 = slim.conv2d(conv2, 128, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv2_3')
        pool2 = slim.avg_pool2d(conv2, [2, 2], padding='SAME')
        conv3 = slim.conv2d(pool2, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_1')
        conv3 = slim.conv2d(conv3, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_2')
        conv3 = slim.conv2d(conv3, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_3')
        conv3 = slim.conv2d(conv3, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_4')
        conv3 = slim.conv2d(conv3, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_5')
        conv3 = slim.conv2d(conv3, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_6')
        conv3 = slim.conv2d(conv3, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_7')#
        conv3 = slim.conv2d(conv3, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_9')#
        conv3 = slim.conv2d(conv3, 256, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv3_10')#
        up4 = upsample_and_sum(conv3, conv2, 128, 256, scope='deconv4')
        conv4 = slim.conv2d(up4, 128, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv4_1')
        conv4 = slim.conv2d(conv4, 128, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv4_2')
        conv4 = slim.conv2d(conv4, 128, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv4_3')
        up5 = upsample_and_sum(conv4, conv1, 64, 128, scope='deconv5')
        conv5 = slim.conv2d(up5, 64, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv5_1')
        conv5 = slim.conv2d(conv5, 64, [3, 3], rate=1, activation_fn=tf.nn.relu, scope='conv5_2')
        out = slim.conv2d(conv5, 1, [1, 1], rate=1, activation_fn=None, scope='conv6')
        return out

def dncnn(input,is_training=True):
    noise_level = FCN(input)
    concat_img = tf.concat([input, noise_level], 3)
    out = UNet(concat_img) + input
    return out

class denoiser(object):
    def __init__(self, sess, sigma, cost_str, ckpt_dir, sample_dir, log_dir):
        self.sess = sess
        self.sigma = sigma
        
        self.ckpt_dir = ckpt_dir
        self.sample_dir = sample_dir
        self.log_dir = log_dir
        
        if not os.path.exists(self.sample_dir):
            os.makedirs(self.sample_dir)
    
        # build model
        #placeholders for clean and noisy image batches
     #   self.GT = tf.placeholder(tf.float32, [None, None, None, 1], name='gt_true_image')
        self.Y = tf.placeholder(tf.float32, [None, None, None, 1], name='noisy_image')
       
        self.is_training = tf.placeholder(tf.bool, name='is_training') #for batchnorm

        #forward propagation
        with tf.variable_scope('DnCNN'):
            self.Y_ = dncnn(self.Y, is_training=self.is_training)

        batch = tf.to_float(tf.shape(self.Y)[0])    #size of the minibatch
        

        self.noisy = self.Y_ - self.Y

        self.Tv_y,_  = tf.image.image_gradients(self.noisy)
        _,self.Tv_x  = tf.image.image_gradients(self.Y_)

        self.Tv_x = 5e4*tf.reduce_mean(tf.abs(self.Tv_x)) #Along the noise direction　#tf.reduce_mean(total_variation_x(self.noisy))
        self.Tv_y = 3e5*tf.reduce_mean(tf.abs(self.Tv_y)) #Vertical noise direction #tf.reduce_mean(total_variation_y(self.Y_))
        #dy_true, dx_true = tf.image.image_gradients(y_true)
        #dy_pred, dx_pred = tf.image.image_gradients(y_pred)

        # MSE
        self.mse = (1.0 / batch)*(tf.nn.l2_loss(self.Y - self.Y_) + self.Tv_x + self.Tv_y)


        #self.sure     = (1.0 / batch)*(tf.nn.l2_loss(self.Y - self.Y_) - self.var_sum_Y + self.divergence_sum_Y + 1e4*tf.reduce_mean(tf.square(tf.image.image_gradients(self.Y_))))
        self.cost = self.mse

        
        self.lr = tf.placeholder(tf.float32, name='learning_rate')
        self.eva_psnr = tf.placeholder(tf.float32, name='eva_psnr')

        #optimizer
        optimizer = tf.train.AdamOptimizer(self.lr, name='AdamOptimizer')

        #for batchnorm
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            self.train_op = optimizer.minimize(self.cost) ##CHANGED

        #checkpoint saver
        self.saver = tf.train.Saver(max_to_keep=15)

        init = tf.global_variables_initializer()
        self.sess.run(init)
        print("[*] Initialize model successfully...")

   #function to evaluate the performance after each epoch

    def train(self, data_path, eval_files, batch_size, epoch_, lr, gt_type):
        
        # CLEAN ground-truth trainset
     #   gt_data= np.load('/content/drive/MyDrive/SURE_pstm/pstm_xiaobo.npy')
        # NOISY patches trainset
        #noisy_name   ='rgb_noisy_' + str(gt_type) + '_patches.npy'
        #noisy        = np.load(os.path.join(data_path, noisy_name), mmap_mode='r')    
        noisy = np.load('/content/drive/MyDrive/yyh/feature_select/test/kerry_440_400_50_1_W.npy')

        numData = np.shape(noisy)[0]
        numBatch = int(numData / batch_size)

        # load pretrained model
       # load_model_status, global_step = self.load(self.ckpt_dir) 
        load_model_status = False
        if load_model_status:
            iter_num = global_step
            start_epoch = (global_step) // numBatch
            start_step = (global_step) % numBatch
            print("[*] Model restore success!")
        else:
            iter_num    = 0
            start_epoch = 0
            start_step  = 0
            print("[*] Not find pretrained model!")

        # make summary
        tf.summary.scalar('MSE', self.mse)
        #tf.summary.scalar('SURE', self.sure)
        tf.summary.scalar('lr', self.lr)
        writer = tf.summary.FileWriter(self.log_dir+"/", self.sess.graph)
        merged = tf.summary.merge_all()
        summary_psnr = tf.summary.scalar('eva_psnr', self.eva_psnr)
        print("[*] Start training, with start epoch %d start iter %d : " % (start_epoch, iter_num))
        start_time = time.time()
       # self.evaluate(eval_files, iter_num, summary_merged=summary_psnr, summary_writer=writer)
        tf.get_default_graph().finalize() # making sure that the graph is fixed at this point
        #training loop
        for epoch in range(start_epoch, epoch_):
            #print("Model: %s" % (self.ckpt_dir))
            #print("Learning rate: {}".format(lr[epoch]))
            
            rand_inds=np.random.choice(numData, numData,replace=False)

            for batch_id in range(0, numBatch):
                # No RAM required
                batch_rand_inds = rand_inds[batch_id * batch_size:(batch_id + 1) * batch_size]
            #    batch_images            = np.array(gt_data[batch_rand_inds]).astype(np.float32) #/ 255.0     # clean gt
                batch_images_corrupt    = np.array(noisy[batch_rand_inds]).astype(np.float32) #/ 255.0       # noisy
                feed_dict = {#self.GT: batch_images,
                             self.Y: batch_images_corrupt,
                             self.lr: lr[epoch],
                             self.is_training: True}
                self.sess.run(self.train_op, feed_dict=feed_dict)
                
                if (iter_num)%100==0:
                    feed_dict2 = {#self.GT: batch_images,
                                 self.Y: batch_images_corrupt,
                                 self.lr: lr[epoch],
                                 self.is_training: False}

                    mse,summary  = self.sess.run([self.mse,merged],feed_dict=feed_dict2)
                    print("Epoch: [%2d] [%4d/%4d] Time: %4.4f"
                          % (epoch + 1, batch_id + 1, numBatch, time.time() - start_time))
                    writer.add_summary(summary, iter_num)

                iter_num += 1
            if epoch==epoch_-1:
                self.save(iter_num, self.ckpt_dir)
            print('\n')
        print("[*] Finish training.")

    def save(self, iter_num, ckpt_dir, model_name='CDnCNN-tensorflow'):
        checkpoint_dir = ckpt_dir
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        print("[*] Saving model...")
        self.saver.save(self.sess,
                   os.path.join(checkpoint_dir, model_name),
                   global_step=iter_num)

    def load(self, checkpoint_dir):
        print("[*] Reading checkpoint...")
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            full_path = tf.train.latest_checkpoint(checkpoint_dir)
            global_step = int(full_path.split('/')[-1].split('-')[-1])
            self.saver.restore(self.sess, full_path)
            return True, global_step
        else:
            return False, 0
          
    def test(self, save_dir):
        """Test CDnCNN"""
        # init variables
        tf.initialize_all_variables().run()

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        
        load_model_status, global_step = self.load(self.ckpt_dir)
        assert load_model_status == True, '[!] Load weights FAILED...'
        print("[*] Load weights SUCCESS...")
       
      #  clean = np.load("/content/drive/MyDrive/SURE_pstm/pstm_xiaobo.npy")
      #  clean_image = clean[0:421,:,:,:]
        noisy = np.load("/content/drive/MyDrive/yyh/feature_select/test/kerry_440_400_50_1_W.npy")
        print(noisy.shape)#(440, 400, 50, 1)
        
        #scio.savemat('noise.mat',{'ori_':noise})
        
        noise = np.reshape(noisy[0:440,0:400,0:50,0],[440,400,50])
        recover = np.zeros((440,400,50))
        j=0
        for i in range(0,440,440): #440
            noisy_image = noisy[i:i+440,:,:,:]
            output_clean_image = self.sess.run(self.Y_, feed_dict={self.Y: noisy_image, self.is_training: False})
            recover[j:j+440,:,:]= output_clean_image[:,:,:,0]
            j = j+440
        scio.savemat('/content/drive/MyDrive/yyh/feature_select/test/recover_440_400_50_W.mat',{'recover':recover})
        noisy = noisy[:,:,:,0]
        print(cal_psnr(recover, noisy))
        print(cal_psnr(noisy, recover))




#　Adjustment parameters
parser = argparse.ArgumentParser(description='') 
parser.add_argument('--epoch', dest='epoch', type=int, default=15, help='# of epoch')
parser.add_argument('--batch_size', dest='batch_size', type=int, default=1, help='# images in batch')
parser.add_argument('--lr', dest='lr', type=float, default=0.0001, help='initial learning rate for sgd')#0.001
parser.add_argument('--sigma', dest='sigma', type=float, default=10.0, help='noise level (for evaluation and testing)')


parser.add_argument('--data', dest='data', default='/content/drive/MyDrive/SURE_htins/', help='training data path')
parser.add_argument('--checkpoint_dir', dest='ckpt_dir', default='/content/drive/MyDrive/SURE_htins/checkpoint', help='models are saved here')
parser.add_argument('--sample_dir', dest='sample_dir', default='./sample', help='sample are saved here')
parser.add_argument('--log_dir', dest='log_dir', default='./logs', help='tensorboard logs are saved here')
parser.add_argument('--test_dir', dest='test_dir', default='./test', help='test sample are saved here')
parser.add_argument('--eval_set', dest='eval_set', default='CBSD68', help='dataset for eval in training')
parser.add_argument('--test_set', dest='test_set', default='CBSD68', help='dataset for testing')

parser.add_argument('--cost', dest='cost', default='SURE', help='cost to minimize: MSE, SURE, e-SURE')
parser.add_argument('--gpu', dest='gpu', default='0', help='which gpu to use')
parser.add_argument('--type', dest='type', default='blind_50_gt10', help='arg to give unique names to realizations')
parser.add_argument('--gt_type', dest='gt_type', default='5', help='arg to give unique names to realizations')
parser.add_argument('--phase', dest='phase', default='test', help='train or test')
parser.add_argument('--use_gpu', dest='use_gpu', type=int, default=1, help='gpu flag, 1 for GPU and 0 for CPU')
args = parser.parse_known_args()[0]


def denoiser_train(denoiser, lr):
    eval_files = glob('/content/drive/MyDrive/Extended_SURE/Dataset/testset/{}/*.png'.format(args.eval_set))
    denoiser.train(args.data, eval_files, batch_size=args.batch_size, epoch_=args.epoch, lr=lr, gt_type=args.gt_type)


def denoiser_test(denoiser, save_dir):

    denoiser.test(save_dir)


def main(_):
    
    #the following string is attached to checkpoint, log and image folder names
    name = "CDnCNN_" + args.cost + '_' + str(args.type)
    
    ckpt_dir = args.ckpt_dir + "/" + name
    sample_dir = args.sample_dir + "/" + name
    test_dir = args.test_dir + "/" + name
    log_dir = args.log_dir + "/" + name
    print('CKPT path: ', ckpt_dir)
    
    if not os.path.exists(args.ckpt_dir):
        os.makedirs(args.ckpt_dir)

    lr = args.lr * np.ones([args.epoch])
    lr[40:] = lr[0] / 10.0 #lr decay

    
    if args.use_gpu:
        # added to control the gpu memory
        print("GPU\n")
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
        with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
            model = denoiser(sess, sigma=args.sigma, cost_str=args.cost, ckpt_dir=ckpt_dir, sample_dir=sample_dir, log_dir=log_dir)
            if args.phase == 'train':
                denoiser_train(model, lr=lr)
            elif args.phase == 'test':
                denoiser_test(model, test_dir)
            else:
                print('[!]Unknown phase')
                exit(0)
    else:
        print("CPU\n")
        with tf.Session() as sess:
            model = denoiser(sess, sigma=args.sigma, cost_str=args.cost, ckpt_dir=ckpt_dir, sample_dir=sample_dir, log_dir=log_dir)
            if args.phase == 'train':
                denoiser_train(model, lr=lr)
            elif args.phase == 'test':
                denoiser_test(model, test_dir)
            else:
                print('[!]Unknown phase')
                exit(0)


if __name__ == '__main__':
    os.environ['CUDA_VISIBLE_DEVICES']=str(args.gpu)
    tf.reset_default_graph()
    tf.app.run()



CKPT path:  /content/drive/MyDrive/SURE_htins/checkpoint/CDnCNN_SURE_blind_50_gt10
GPU

[*] Initialize model successfully...


I1019 10:36:58.247167 139640794634112 saver.py:1412] Restoring parameters from /content/drive/MyDrive/SURE_htins/checkpoint/CDnCNN_SURE_blind_50_gt10/CDnCNN-tensorflow-6600


[*] Reading checkpoint...
[*] Load weights SUCCESS...
(440, 400, 50, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


21.219205123123107
21.06298211526681


SystemExit: ignored